# Data Cleaning and EDA

In [ ]:
# double-check requirements
!pip install -r requirements.txt

In [3]:
import polars as pl
import pandas as pd
import os
import re

In [4]:
# Establish root path of the project
root_path = os.getcwd()

## Reference: Data Dictionary

In [5]:
data_dict = pl.read_csv(root_path + "/data/data_dictionary.csv")
data_dict

## HBN Internet Usage Data
**This will be denoted as `iuse_data`**

Information about HBN Internet Usage data
- `Demographics` - Information about age and sex of participants.
- `Internet Use` - Number of hours of using computer/internet per day.
- `Children's Global Assessment Scale` - Numeric scale used by mental health clinicians to rate the general functioning of youths under the age of 18.
- `Physical Measures` - Collection of blood pressure, heart rate, height, weight and waist, and hip measurements.
- `FitnessGram Vitals and Treadmill` - Measurements of cardiovascular fitness assessed using the NHANES treadmill protocol.
- `FitnessGram Child` - Health related physical fitness assessment measuring five different parameters including aerobic capacity, muscular strength, muscular endurance, flexibility, and body composition.
- `Bio-electric Impedance Analysis` - Measure of key body composition elements, including BMI, fat, muscle, and water content.
- `Physical Activity Questionnaire` - Information about children's participation in vigorous activities over the last 7 days.
- `Sleep Disturbance Scale` - Scale to categorize sleep disorders in children.
- `Actigraphy` - Objective measure of ecological physical activity through a research-grade biotracker.
- `Parent-Child Internet Addiction Test` - 20-item scale that measures characteristics and behaviors associated with compulsive use of the Internet including compulsivity, escapism, and dependency.

In [56]:
# Read in data
df = pl.read_csv(root_path + "/data/train.csv")
df.head()


id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,…,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,PCIAT-Season,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
str,str,i64,i64,str,i64,str,f64,f64,f64,f64,i64,i64,i64,str,i64,i64,i64,str,i64,i64,f64,i64,f64,i64,i64,i64,f64,i64,f64,i64,f64,i64,str,i64,f64,f64,…,f64,f64,f64,f64,f64,str,f64,str,f64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,str,i64,i64
"""00008ff9""","""Fall""",5,0,"""Winter""",51,"""Fall""",16.877316,46.0,50.8,null,null,null,null,null,null,null,null,"""Fall""",0,0,null,null,null,null,0,0,7.0,0,6.0,0,6.0,1,"""Fall""",2,2.66855,16.8792,…,24.4349,8.89536,38.9177,19.5413,32.6909,null,null,null,null,"""Fall""",5,4,4,0,4,0,0,4,0,0,4,0,4,4,4,4,4,4,2,4,55,null,null,null,"""Fall""",3,2
"""000fd460""","""Summer""",9,0,null,null,"""Fall""",14.03559,48.0,46.0,22.0,75,70,122,null,null,null,null,"""Fall""",3,0,null,null,null,null,5,0,11.0,1,11.0,1,3.0,0,"""Winter""",2,2.57949,14.0371,…,21.0352,14.974,39.4497,15.4107,27.0552,null,null,"""Fall""",2.34,"""Fall""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""Fall""",46,64,"""Summer""",0,0
"""00105258""","""Summer""",10,1,"""Fall""",71,"""Fall""",16.648696,56.5,75.6,null,65,94,117,"""Fall""",5,7,33,"""Fall""",20,1,10.2,1,14.7,2,7,1,10.0,1,10.0,1,5.0,0,null,null,null,null,…,null,null,null,null,null,null,null,"""Summer""",2.17,"""Fall""",5,2,2,1,2,1,1,2,1,1,1,0,1,1,1,0,2,2,1,1,28,"""Fall""",38,54,"""Summer""",2,0
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,30.4041,16.779,58.9338,26.4798,45.9966,null,null,"""Winter""",2.451,"""Summer""",4,2,4,0,5,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1
"""0016bb22""","""Spring""",18,1,"""Summer""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,"""Summer""",1.04,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [57]:
# Per columns category, get the corresponding columns so that we can analyze per category differences
matches = [re.search(r"([A-Za-z_]*)?-", col) for col in df.columns]
instrument_categories = list(pl.Series([match.group(1) if match != None else None for match in matches]).unique().drop_nulls())

def get_corresponding_cols(instrument_category):
    return set(pl.Series([col if re.search(r"([A-Za-z_]*)?-", col) != None and re.search(r"([A-Za-z_]*)?-", col).group(1) == instrument_category else None for col in df.columns]).drop_nulls())

# Create a dictionary of `instrument_category`: `set(corresponding fields)
categories_to_columns = {category : get_corresponding_cols(category) for category in instrument_categories}
categories_to_columns

{'PAQ_A': {'PAQ_A-PAQ_A_Total', 'PAQ_A-Season'},
 'Fitness_Endurance': {'Fitness_Endurance-Max_Stage',
  'Fitness_Endurance-Season',
  'Fitness_Endurance-Time_Mins',
  'Fitness_Endurance-Time_Sec'},
 'FGC': {'FGC-FGC_CU',
  'FGC-FGC_CU_Zone',
  'FGC-FGC_GSD',
  'FGC-FGC_GSD_Zone',
  'FGC-FGC_GSND',
  'FGC-FGC_GSND_Zone',
  'FGC-FGC_PU',
  'FGC-FGC_PU_Zone',
  'FGC-FGC_SRL',
  'FGC-FGC_SRL_Zone',
  'FGC-FGC_SRR',
  'FGC-FGC_SRR_Zone',
  'FGC-FGC_TL',
  'FGC-FGC_TL_Zone',
  'FGC-Season'},
 'Basic_Demos': {'Basic_Demos-Age',
  'Basic_Demos-Enroll_Season',
  'Basic_Demos-Sex'},
 'PAQ_C': {'PAQ_C-PAQ_C_Total', 'PAQ_C-Season'},
 'BIA': {'BIA-BIA_Activity_Level_num',
  'BIA-BIA_BMC',
  'BIA-BIA_BMI',
  'BIA-BIA_BMR',
  'BIA-BIA_DEE',
  'BIA-BIA_ECW',
  'BIA-BIA_FFM',
  'BIA-BIA_FFMI',
  'BIA-BIA_FMI',
  'BIA-BIA_Fat',
  'BIA-BIA_Frame_num',
  'BIA-BIA_ICW',
  'BIA-BIA_LDM',
  'BIA-BIA_LST',
  'BIA-BIA_SMM',
  'BIA-BIA_TBW',
  'BIA-Season'},
 'CGAS': {'CGAS-CGAS_Score', 'CGAS-Season'},
 'PreIn

In [87]:
# Basic summary statistics about the makeup of the data
# Understanding Missingness
def get_category(text):
    return re.search(r"([A-Za-z_]*)?-", text)
with pl.Config(tbl_rows=50):
    print(df.with_columns(
        pl.col("*").is_null() / df.shape[0]
    ).sum().with_columns(
        pl.col("*").round(3)
    ).transpose(include_header = True, header_name = "fields", column_names = ["prop_null"]).with_columns(
        pl.col("fields").str.extract(r"([A-Za-z_]*)?-", 1).alias("category")
    ).drop_nulls().group_by(pl.col('category')).agg(pl.col("prop_null").mean().round(3)).sort(pl.col('prop_null'), descending=True))

shape: (11, 2)
┌───────────────────┬───────────┐
│ category          ┆ prop_null │
│ ---               ┆ ---       │
│ str               ┆ f64       │
╞═══════════════════╪═══════════╡
│ PAQ_A             ┆ 0.88      │
│ Fitness_Endurance ┆ 0.777     │
│ PAQ_C             ┆ 0.565     │
│ BIA               ┆ 0.495     │
│ FGC               ┆ 0.486     │
│ CGAS              ┆ 0.372     │
│ SDS               ┆ 0.341     │
│ PCIAT             ┆ 0.31      │
│ Physical          ┆ 0.299     │
│ PreInt_EduHx      ┆ 0.136     │
│ Basic_Demos       ┆ 0.0       │
└───────────────────┴───────────┘


## HBN Physical Activity Data
**This will be denoted as `phys_data`**

`series_{train|test}.parquet/id={id}` - Series to be used as training data, partitioned by id. Each series is a continuous recording of accelerometer data for a single subject spanning many days.

- `id` - The patient identifier corresponding to the id field in train/test.csv.
- `step` - An integer timestep for each observation within a series.
- `X, Y, Z` - Measure of acceleration, in g, experienced by the wrist-worn watch along each standard axis.
- `enmo` - As calculated and described by the wristpy package, ENMO is the Euclidean Norm Minus One of all accelerometer signals (along each of the x-, y-, and z-axis, measured in g-force) with negative values rounded to zero. Zero values are indicative of periods of no motion. While no standard measure of acceleration exists in this space, this is one of the several commonly computed features.
- `anglez` - As calculated and described by the wristpy package, Angle-Z is a metric derived from individual accelerometer components and refers to the angle of the arm relative to the horizontal plane.
- `non-wear_flag` - A flag (0: watch is being worn, 1: the watch is not worn) to help determine periods when the watch has been removed, based on the GGIR definition, which uses the standard deviation and range of the accelerometer data.
- `light` - Measure of ambient light in lux. See here for [details](https://actigraphcorp.my.site.com/support/s/article/Lux-Measurements).
- `battery_voltage` - A measure of the battery voltage in mV.
- `time_of_day` - Time of day representing the start of a 5s window that the data has been sampled over, with format %H:%M:%S.%9f.
- `weekday` - The day of the week, coded as an integer with 1 being Monday and 7 being Sunday.
- `quarter` - The quarter of the year, an integer from 1 to 4.
- `relative_date_PCIAT` - The number of days (integer) since the PCIAT test was administered (negative days indicate that the actigraphy data has been collected before the test was administered).

In [24]:
import glob
import re

In [18]:
series_train_list = glob.glob(root_path + "/data/series_train.parquet/*/*")
series_test_list = glob.glob(root_path + "/data/series_test.parquet/*/*")

In [26]:
text = series_train_list[0]

In [38]:
# Pattern to extract uid for later merges (after we post-process location data)
out = re.search(r"id=([0-9a-z]*)?/", text)
uid = out.group(1)
uid

'5b2c9b83'

In [22]:
pd.read_parquet(series_train_list[0])

,step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,time_of_day,weekday,quarter,relative_date_PCIAT
0,0,0.066507,-0.037770,-0.998841,0.004343,-85.734299,0.0,13.000,4185.000000,40740000000000,1,1,160.0
1,1,0.066573,-0.037481,-0.998446,0.002900,-85.613945,0.0,14.000,4185.083496,40745000000000,1,1,160.0
2,2,0.066664,-0.037573,-0.997421,0.003198,-85.611862,0.0,14.125,4185.166504,40750000000000,1,1,160.0
3,3,0.218984,-0.063940,-0.917868,0.151349,-70.192177,0.0,14.250,4185.250000,40755000000000,1,1,160.0
4,4,0.325113,0.099378,-0.931720,0.026295,-69.375443,0.0,14.375,4185.333496,40760000000000,1,1,160.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
390043,390043,-0.013766,0.070419,1.022340,0.024868,85.979820,1.0,16.000,3098.833252,3755000000000,3,1,183.0
390044,390044,-0.013949,0.070235,1.022550,0.025065,85.958809,1.0,16.000,3098.666748,3760000000000,3,1,183.0
390045,390045,-0.013897,0.070209,1.022813,0.025326,85.989395,1.0,16.000,3098.500000,3765000000000,3,1,183.0
390046,390046,-0.013988,0.070077,1.022510,0.025015,85.965820,1.0,16.000,3098.333252,3770000000000,3,1,183.0


In [ ]:
import matplotlib.pyplot as plt